## Assigmnent 3

In [1]:
from PattRecClasses import DiscreteD, GaussD, HMM, MarkovChain
from matplotlib import pyplot as plt

import os
import numpy as np

In [2]:
#https://www.kaggle.com/datasets/avk256/activity-recognition

### Feature extraction

In [3]:

acc = []
#tau = [] # sample index for first sample in each recording

for filename in os.listdir('sensorlogger'):
    path = os.path.join('sensorlogger',filename)
    with open(path) as file:        
        if os.path.isfile(path):
            lines = [line.split() for line in file]
            # abs of norm of acceleration - gravity
            acc.append([ np.abs((np.sqrt(float(line[2])**2 + float(line[3])**2 + float(line[4])**2))-9.82) for line in lines ])
            #tau.append(len(acc)+1)
        
        
#t = [(float(line[0])-float(lines[0][0]))/1000 for line in lines]



In [4]:
%matplotlib qt
plt.figure(figsize= (15,10))
plt.subplot(311)
plt.plot(acc[0])

"""Windowing"""
wLen = 100 # window length (samples)
overlap = 0 # is higher resolution needed?
windows = []

step = wLen-overlap

X = []
for rec in acc:
    x = []
    wnd = []
    for n in range(0,len(rec)-wLen +1,step):
        x.append( sum(rec[n:n+wLen])/wLen )
        wnd.extend( [sum(rec[n:n+wLen])/wLen] *wLen )

    print(np.shape(x))
    X.append(x)
    windows.append(wnd)

    
"""Labels"""
S = []

"""Cluster training data into N separate categories, for estimation of output probabilities"""
data1 = []
data2 = []
data3 = []

for rec in X:
    s = []
    for a in rec:
        if a > 3.5:
            s.append(3) #running
            data3.append(a)
        elif a > 0.6:
            s.append(2)  #walking
            data2.append(a)
        else:
            s.append(1)  #standing
            data1.append(a)
            
    S.append(s)
        
colors = {1: 'red', 2: 'green', 3: 'blue'}

plt.subplot(312)
plt.plot(windows[0])

plt.subplot(313)
plt.plot([t2 for t2 in range(0, len(acc[0]), wLen)][:-1], X[0])
for i, val in enumerate(S[0]):
    plt.scatter(i*wLen, val, color = colors[val])

plt.show()


/nix/store/b2hc0i92l22ir2kavnjn3z5z6mzabbvm-glibc-2.34-210/lib/libc.so.6: version `GLIBC_2.38' not found (required by /nix/store/6j6hl9x71gpq4siar7ns8w2fwx0s7d6q-gvfs-1.52.2/lib/gio/modules/libgvfsdbus.so)
Failed to load module: /nix/store/6j6hl9x71gpq4siar7ns8w2fwx0s7d6q-gvfs-1.52.2/lib/gio/modules/libgvfsdbus.so
QApplication: invalid style override 'gtk2' passed, ignoring it.
	Available styles: bb10dark, bb10bright, cleanlooks, gtk2, cde, motif, plastique, Windows, Fusion
qt.glx: qglx_findConfig: Failed to finding matching FBConfig for QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBufferSize -1, redBufferSize 1, greenBufferSize 1, blueBufferSize 1, alphaBufferSize -1, stencilBufferSize -1, samples -1, swapBehavior QSurfaceFormat::SingleBuffer, swapInterval 1, colorSpace QSurfaceFormat::DefaultColorSpace, profile  QSurfaceFormat::NoProfile)
No XVisualInfo for format QSurfaceFormat(version 2.0, options QFlags<QSurfaceFormat::FormatOption>(), depthBuff

(233,)
(6,)
(161,)
(176,)
(310,)
(104,)
(150,)


In [5]:
"""

fig, (ax1,ax2,ax3) = plt.subplots(3)

ax1.hist(data1, bins = 30)
ax2.hist(data2, bins = 30)
ax3.hist(data3, bins = 30)
plt.show()
"""

'\n\nfig, (ax1,ax2,ax3) = plt.subplots(3)\n\nax1.hist(data1, bins = 30)\nax2.hist(data2, bins = 30)\nax3.hist(data3, bins = 30)\nplt.show()\n'

In [6]:
"""Extracting data to csv"""

datapoints = np.array([X,S]).T

import csv

with open('features.csv', 'w') as csvfile:
    Writer = csv.writer(csvfile)
    for data in datapoints:
        Writer.writerow(data)



/tmp/nix-shell.QMcQW7/ipykernel_7527/2181927254.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  datapoints = np.array([X,S]).T


NameError: name 'tau' is not defined

In [7]:
"""Initialization"""

"""Infinite ergodic HMM may start in any state. Assumption here is that
    each state is equally likely to occur as initial state"""

mc = MarkovChain( np.full( (1,3),1/3 ) , np.array( [ [ 0.9, 0.09, 0.01 ], 
                                                  [ 0.05, 0.9, 0.05 ], 
                                                  [ 0.01, 0.09 ,0.9] ] ) )

"""The diagonal elements of the transition matrix can be chosen to indicate 
    approximately the average state durations. Here I assume that each state duration is equally long"""


"""Output probability distributions must be initialized differently for each state"""
# Maybe do this with separate data?

from scipy.stats import norm
        
mu1, std1 = norm.fit(data1) #standing
mu2, std2 = norm.fit(data2) #walking
mu3, std3 = norm.fit(data3) #running


# Looks reasonable
print(mu1,std1)
print(mu2,std2)
print(mu3,std3)


b1 = GaussD( means=[mu1], stdevs=[std1] )   # Distribution for state = standing
b2 = GaussD( means=[mu2], stdevs=[std2] )   # Distribution for state = walking
b3 = GaussD( means=[mu3], stdevs=[std3] )   # Distribution for state = running

"""Initial HMM"""

h0  = HMM( mc, [b1, b2, b3])

0.2149082219447096 0.12722765619672527
1.7420355493184476 0.45829907888362154
5.9660427944252 0.9862693139955239


### Training

In [43]:
"""Updating initial probability vector"""

epsilon = 10e-10


gma = []
# for x in training data
for x_obs in X:
    pX = np.array([b1.prob(x_obs), b2.prob(x_obs), b3.prob(x_obs)])
    

    # replace zero probabilities with some small probability epsilon
    with np.nditer(pX, op_flags =['readwrite']) as it:
        for x in it:
            x[...] = max(x,epsilon)

    """normalize"""
    pX = pX / np.sum(pX,axis=0)

    #sf = [np.max(p) for p in pX.T]
    #pX = pX/sf

    [alfaHat, c] = mc.forward(pX)
    betaHat = mc.backward(pX, c)

    tmp = np.multiply(alfaHat, betaHat)
    gma_r = np.multiply(tmp,c) # gamma i,t
    gma.append(gma_r)
        
"""Update equation"""
q_new = np.zeros((1,3))
for r in gma:
    q_new = q_new + r[:,0]
q_new = q_new/len(gma)

print(q_new)

[[0.4285534  0.29092337 0.28052323]]


In [ ]:
"""Updating transition probability matrix"""


"""The factors ξij,t express the conditional probability, given all observations, 
   that the hidden state was St = i ∩ St+1 =j"""


